In [1]:
import numpy as np
import cupy as cp
import skimage.io
import skimage.color
import matplotlib.pyplot as plt
from skimage.transform import downscale_local_mean
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from numba import njit
import time


In [2]:
import pandas as pd
import numpy as np

def salvar_resultados_em_xlsx(lista_dicionarios, caminho_arquivo='resultados.xlsx'):
    # Converte os dicionários para DataFrame
    df = pd.DataFrame(lista_dicionarios)
    
    # Converte todos os np.float64 para float padrão do Python
    df = df.applymap(lambda x: float(x) if isinstance(x, np.float64) else x)
    
    # Salva como arquivo Excel
    df.to_excel(caminho_arquivo, index=False)
    print(f'Arquivo salvo em: {caminho_arquivo}')

In [3]:
@njit
def mirror_cpu(A, f):
    n, m = A.shape
    nlin = n + 2*f
    ncol = m + 2*f
    B = np.zeros((nlin, ncol), dtype=A.dtype)

    B[f:n+f, f:m+f] = A
    B[0:f, f:m+f] = A[0:f, :][::-1, :]
    B[n+f:, f:m+f] = A[n-f:, :][::-1, :]
    B[f:n+f, 0:f] = A[:, 0:f][:, ::-1]
    B[f:n+f, m+f:] = A[:, m-f:][:, ::-1]
    B[0:f, 0:f] = A[0:f, 0:f][::-1, ::-1]
    B[0:f, m+f:] = A[0:f, m-f:][::-1, ::-1]
    B[n+f:, 0:f] = A[n-f:, 0:f][::-1, ::-1]
    B[n+f:, m+f:] = A[n-f:, m-f:][::-1, ::-1]

    return B


In [4]:
@njit
def NLM_fast_cpu(img, h, f, t):
    m, n = img.shape
    filtrada = np.zeros((m, n), dtype=img.dtype)
    img_n = mirror_cpu(img, f)
    for i in range(m):
        for j in range(n):
            im = i + f
            jn = j + f
            W1 = img_n[im-f:im+f+1, jn-f:jn+f+1]
            rmin = max(im-t, f)
            rmax = min(im+t, m+f-1)
            smin = max(jn-t, f)
            smax = min(jn+t, n+f-1)
            NL = 0.0
            Z = 0.0
            for r in range(rmin, rmax+1):
                for s in range(smin, smax+1):
                    W2 = img_n[r-f:r+f+1, s-f:s+f+1]
                    d2 = np.sum((W1 - W2)**2)
                    sij = np.exp(-d2/(h**2))
                    Z += sij
                    NL += sij * img_n[r, s]
            filtrada[i, j] = NL / Z
    return filtrada


In [5]:
def mirror_gpu(A, f):
    n, m = A.shape
    B = cp.zeros((n + 2*f, m + 2*f), dtype=A.dtype)
    B[f:n+f, f:m+f] = A
    B[0:f, f:m+f] = A[0:f, :][::-1, :]
    B[n+f:, f:m+f] = A[n-f:, :][::-1, :]
    B[f:n+f, 0:f] = A[:, 0:f][:, ::-1]
    B[f:n+f, m+f:] = A[:, m-f:][:, ::-1]
    B[0:f, 0:f] = A[0:f, 0:f][::-1, ::-1]
    B[0:f, m+f:] = A[0:f, m-f:][::-1, ::-1]
    B[n+f:, 0:f] = A[n-f:, 0:f][::-1, ::-1]
    B[n+f:, m+f:] = A[n-f:, m-f:][::-1, ::-1]
    return B


In [6]:
nlm_kernel_global_code = r'''
extern "C" __global__
void nlm_kernel_global(
    const float* img_n, float* output,
    int m, int n, int f, int t, float h, int padded_width
) {
    int i = blockIdx.y * blockDim.y + threadIdx.y;
    int j = blockIdx.x * blockDim.x + threadIdx.x;

    if (i >= m || j >= n)
        return;

    int im = i + f;
    int jm = j + f;

    float NL = 0.0f;
    float Z = 0.0f;

    for (int r = im - t; r <= im + t; ++r) {
        for (int s = jm - t; s <= jm + t; ++s) {
            float d2 = 0.0f;
            for (int u = -f; u <= f; ++u) {
                for (int v = -f; v <= f; ++v) {
                    int x1 = im + u;
                    int y1 = jm + v;
                    int x2 = r + u;
                    int y2 = s + v;

                    float val1 = img_n[x1 * padded_width + y1];
                    float val2 = img_n[x2 * padded_width + y2];
                    float diff = val1 - val2;
                    d2 += diff * diff;
                }
            }
            float weight = __expf(-d2 / (h * h));
            Z += weight;
            NL += weight * img_n[r * padded_width + s];
        }
    }

    output[i * n + j] = NL / Z;
}
'''



In [7]:
import sklearn.neighbors as sknn
import networkx as nx

def process_pixel(i, j, img_n, f, t, h, nn, m, n):
    im = i + f
    jn = j + f
    patch_central = img_n[im - f:im + f + 1, jn - f:jn + f + 1]
    central = patch_central.ravel()

    rmin = max(im - t, f)
    rmax = min(im + t, m + f)
    smin = max(jn - t, f)
    smax = min(jn + t, n + f)

    n_patches = (rmax - rmin) * (smax - smin)
    patch_size = (2 * f + 1) ** 2

    dataset = np.empty((n_patches, patch_size), dtype=np.float32)
    pixels_busca = np.empty(n_patches, dtype=np.float32)

    source = -1
    k = 0
    for r in range(rmin, rmax):
        for s in range(smin, smax):
            W = img_n[r - f:r + f + 1, s - f:s + f + 1]
            neighbor = W.ravel()
            dataset[k, :] = neighbor
            pixels_busca[k] = img_n[r, s]
            if np.array_equal(central, neighbor):
                source = k
            k += 1

    if source == -1:
        source = 0  # fallback para evitar erro

    knnGraph = sknn.kneighbors_graph(dataset, n_neighbors=nn, mode='distance')
    G = nx.from_scipy_sparse_array(knnGraph)
    length, _ = nx.single_source_dijkstra(G, source)

    points = list(length.keys())
    distancias = np.array(list(length.values()), dtype=np.float32)

    similaridades = np.exp(-distancias ** 2 / (h ** 2))
    pixels = pixels_busca[points]

    NL = np.sum(similaridades * pixels)
    Z = np.sum(similaridades)
    return NL / Z if Z > 0 else img_n[im, jn]


from joblib import Parallel, delayed

def Parallel_GEONLM(img_n, f, t, h, nn):
    print(f'img_n.shape: {img_n.shape}')
    m = img_n.shape[0] - 2 * f
    n = img_n.shape[1] - 2 * f
    print(f'M: {m}, N: {n}')

    filtrada = Parallel(n_jobs=-1)(
        delayed(process_pixel)(i, j, img_n, f, t, h, nn, m, n)
        for i in range(m)
        for j in range(n)
    )

    filtrada_geo = np.array(filtrada).reshape((m, n))
    return filtrada_geo

In [8]:
def NLM_fast_cuda_global(img, h, f, t):
    img = img.astype(cp.float32)
    m, n = img.shape
    padded = mirror_gpu(img, f)

    module = cp.RawModule(code=nlm_kernel_global_code, options=('-std=c++11',))
    kernel = module.get_function("nlm_kernel_global")

    output = cp.zeros((m, n), dtype=cp.float32)
    threads_per_block = (16, 16)
    grid = ((n + 15) // 16, (m + 15) // 16)

    kernel(
        grid, threads_per_block,
        (
            padded.ravel(), output.ravel(),
            cp.int32(m), cp.int32(n), cp.int32(f), cp.int32(t),
            cp.float32(h), cp.int32(padded.shape[1])
        )
    )
    return output

def add_low_noise_gaussian(img):
    m, n = img.shape
    sigma = 5
    noise = np.random.normal(0, sigma, (m, n)).astype(np.float32)
    noised = np.clip(img + noise, 0, 255)
    return noised

def add_moderate_noise_gaussian(img):
    m, n = img.shape
    sigma = 10
    noise = np.random.normal(0, sigma, (m, n)).astype(np.float32)
    noised = np.clip(img + noise, 0, 255)
    return noised

def add_high_noise_gaussian(img):
    m, n = img.shape
    sigma = 10
    noise = np.random.normal(0, sigma, (m, n)).astype(np.float32)
    noised = np.clip(img + noise, 0, 255)
    return noised

def add_poisson_noise(img):    
    poisson_img = np.random.poisson(img)    
    return poisson_img

def add_poisson_gaussian_noise(image, gaussian_sigma=25):
    # Apply Poisson noise
    poisson_image = np.random.poisson(image)

    # Apply Gaussian noise
    gaussian_noise = np.random.normal(loc=0.0, scale=gaussian_sigma, size=image.shape)
    noisy_image = poisson_image + gaussian_noise 

    noisy_image = np.clip(noisy_image, 0, 255).astype(np.uint8)
    
    return noisy_image

def add_salt_and_pepper_noise(image, salt_prob=0.01, pepper_prob=0.01):   
    """
    Adds salt and pepper noise to an image.

    Parameters:
    - image: input image (numpy array).
    - salt_prob: probability of salt pixels (white).
    - pepper_prob: probability of pepper pixels (black).

    Returns:
    - Image with salt and pepper noise.
    """
    
    row, col = image.shape
    noisy = np.copy(image)

    # Adds salt noise
    salt_pixels = np.random.rand(row, col) < salt_prob
    noisy[salt_pixels] = 255

    # Adds pepper noise
    pepper_pixels = np.random.rand(row, col) < pepper_prob
    noisy[pepper_pixels] = 0

    return noisy.astype(np.uint8)

def downscale(img):
    img_downscale = downscale_local_mean(img, (2, 2)).astype(np.float32)
    return img_downscale

def anscombe_transform(img):
    return 2.0 * np.sqrt(img.astype(np.float32) + 3.0 / 8.0)

def inverse_anscombe(y):
    # Inversa exata de Makitalo & Foi (IEEE T-IP 2011)
    return ((y / 2.0)**2) - 3.0 / 8.0

def compute_adaptive_q(sigma_est, anscombe=False):
    q_nlm = 0.8 + 0.5 * np.tanh(0.3 * (sigma_est - 1))
    q_geo = 1.0 + 0.7 * np.tanh(0.25 * (sigma_est - 1.5))

    if anscombe:
        q_nlm = int(np.clip(q_nlm, 0.7, 2.2) * 10)
        q_geo = int(np.clip(q_geo, 0.9, 2.7) * 10)
    else:
        q_nlm = int(np.clip(q_nlm, 0.7, 2.2) * 100)
        q_geo = int(np.clip(q_geo, 0.9, 2.7) * 100)

    print(f'sigma_est: {sigma_est}, q_nlm: {q_nlm}, g_geo: {q_geo}')
    return q_nlm, q_geo

from skimage.metrics import peak_signal_noise_ratio, structural_similarity

def select_best_h_using_adaptive_q(image, image_gpu, q_nlm_candidates, f, t, alpha=0.5, anscombe=False):
    melhor_score = -float('inf')
    melhor_q_nlm = None
    melhor_resultado = None
    melhor_psnr = None
    melhor_ssim = None

    for h_nlm in q_nlm_candidates:
        #print(f'h_nlm: {h_nlm}')

        # Aplica NLM
        result_gpu = NLM_fast_cuda_global(image_gpu, h_nlm, f, t)
        cp.cuda.Stream.null.synchronize()

        # Referência: imagem original
        img_ref = np.clip(image, 0, 255).astype(np.uint8)

        if anscombe:
            # Inversa de Anscombe
            result_processed = np.maximum(inverse_anscombe(cp.asnumpy(result_gpu)), 0)
        else:
            result_processed = cp.asnumpy(result_gpu)

        result_uint8 = np.clip(result_processed, 0, 255).astype(np.uint8)

        # Métricas
        psnr = peak_signal_noise_ratio(img_ref, result_uint8, data_range=255)
        ssim = structural_similarity(img_ref, result_uint8, data_range=255)

        # Score combinado
        score = alpha * psnr + (1 - alpha) * (ssim * 100)
        print(f"h = {h_nlm:.2f} | PSNR = {psnr:.2f} | SSIM = {ssim:.4f} | Score = {score:.2f}")

        if score > melhor_score:
            melhor_score = score
            melhor_q_nlm = h_nlm
            melhor_resultado = result_processed
            melhor_psnr = psnr
            melhor_ssim = ssim

    print(f"\n[SELECIONADO] H = {melhor_q_nlm:.2f} | PSNR = {melhor_psnr:.2f} | SSIM = {melhor_ssim:.4f} | SCORE = {melhor_score:.2f}")

    return melhor_resultado, melhor_q_nlm, melhor_psnr, melhor_ssim, melhor_score


In [9]:
import os
def read_directories(directory, img=None, exclude_json=None):
    # Get a list of filenames in the specified directory
    filenames = []
    for filename in os.listdir(directory):
        if img is not None:
            # If 'img' is provided, filter filenames containing it
            if img in filename:   
                filenames.append(filename)
        elif exclude_json is not None:
            filenames.append(filename.replace('.json',''))     
        else:
            filenames.append(filename)    
    return filenames

### Noise Gaussian

In [10]:
from skimage.restoration import estimate_sigma
import sys


from pathlib import Path

root_dir = f'/workspace/ProjetoDoutorado/Gaussian_Experiments/Moderate_noise/256x256'
dir_images = f'/workspace/ProjetoDoutorado/images'



dir_out_nlm = f'{root_dir}/out_put_NLM'
dir_out_geonlm =  f'{root_dir}/out_put_GNLM'
dir_out_bm3d =  f'{root_dir}/out_put_BM3D'


array_dir = read_directories(dir_images)

array_nln_gaussian_256_filtereds = []

for file in array_dir:

    file_name = file
    img = skimage.io.imread(f'{dir_images}/{file_name}')
    img = img[0, :, :] if len(img.shape) > 2 else img
    if len(img.shape) > 2:
        img = skimage.color.rgb2gray(img)
        img = 255 * img

    img_downscale = downscale(img)
    m, n = img.shape
    ruidosa = add_moderate_noise_gaussian(img_downscale)

    # Clipa imagem para intervalo [0, 255]
    ruidosa[np.where(ruidosa > 255)] = 255
    ruidosa[np.where(ruidosa < 0)] = 0

    # Preparo para CPU e GPU
    img_noisse_gaussian_np = ruidosa.astype(np.float32)
    estimated_sigma_gaussian_np = estimate_sigma(img_noisse_gaussian_np)
    img_gpu_noisse_gaussian = cp.array(img_noisse_gaussian_np)
    

    h_nlm , g_nlm = compute_adaptive_q(estimated_sigma_gaussian_np)

    q_nlm_candidates = np.array([h_nlm + delta for delta in range(-100, 50, 1)])


    img_filtered_nlm, nlm_h, psnr_nlm, ssim_nlm, score_nlm = select_best_h_using_adaptive_q(
        image=img_downscale,
        image_gpu=img_gpu_noisse_gaussian,
        q_nlm_candidates=q_nlm_candidates,
        f=4, t=7,
        alpha=0.5,
        anscombe=False
    )

    dict = {
        'img_noisse_gaussian_np':img_noisse_gaussian_np,
        'img_filtered_nlm':img_filtered_nlm,
        'estimated_sigma_gaussian':estimated_sigma_gaussian_np,
        'nlm_h':nlm_h, 
        'psnr_nlm': psnr_nlm,
        'ssim_nlm':ssim_nlm,
        'score_nlm':score_nlm,
        'file_name':file_name,
    }
    array_nln_gaussian_256_filtereds.append(dict)
    skimage.io.imsave(f'{dir_out_nlm}/{file_name}', img_filtered_nlm.astype(np.uint8))

sigma_est: 11.588373811051037, q_nlm: 129, g_geo: 169
h = 29.00 | PSNR = 28.19 | SSIM = 0.8207 | Score = 55.13
h = 30.00 | PSNR = 28.19 | SSIM = 0.8207 | Score = 55.13
h = 31.00 | PSNR = 28.19 | SSIM = 0.8207 | Score = 55.13
h = 32.00 | PSNR = 28.19 | SSIM = 0.8207 | Score = 55.13
h = 33.00 | PSNR = 28.19 | SSIM = 0.8207 | Score = 55.13
h = 34.00 | PSNR = 28.19 | SSIM = 0.8207 | Score = 55.13
h = 35.00 | PSNR = 28.19 | SSIM = 0.8208 | Score = 55.13
h = 36.00 | PSNR = 28.19 | SSIM = 0.8208 | Score = 55.13
h = 37.00 | PSNR = 28.19 | SSIM = 0.8209 | Score = 55.14
h = 38.00 | PSNR = 28.19 | SSIM = 0.8210 | Score = 55.15
h = 39.00 | PSNR = 28.19 | SSIM = 0.8212 | Score = 55.16
h = 40.00 | PSNR = 28.20 | SSIM = 0.8214 | Score = 55.17
h = 41.00 | PSNR = 28.20 | SSIM = 0.8218 | Score = 55.19
h = 42.00 | PSNR = 28.21 | SSIM = 0.8223 | Score = 55.22
h = 43.00 | PSNR = 28.22 | SSIM = 0.8229 | Score = 55.26
h = 44.00 | PSNR = 28.23 | SSIM = 0.8237 | Score = 55.30
h = 45.00 | PSNR = 28.25 | SSIM = 

In [11]:
import pickle
from pathlib import Path

def save_pickle(obj, path):
    path = Path(path)
    with path.open("wb") as f:
        # usa o protocolo mais moderno disponível
        pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL)

def load_pickle(path):
    path = Path(path)
    with path.open("rb") as f:
        return pickle.load(f)


In [12]:
save_pickle(array_nln_gaussian_256_filtereds, "resultados_gaussian_256.pkl")

### Novo 09/09/25

In [13]:
def run_geonlm_pipeline(img_original, h_base, img_noisy, f, t, mult, nn=10):

    img_noisy_mirror = mirror_cpu(img_noisy, f)    
   
    img_n_geo = np.pad(img_noisy_mirror, ((f, f), (f, f)), 'symmetric')  
   
   
    h_geo = (h_base) * mult
    print(f"\nExecutando GEONLM com h = {h_geo:.2f} (base {h_base} * {mult})")

    img_geo = Parallel_GEONLM(img_n_geo, f=f, t=t, h=h_geo, nn=nn)

    img_geo_no_pad = img_geo[f:-f, f:-f]  # Remove 'f' pixels de cada lado
    img_geo_no_pad = np.clip(img_geo_no_pad, 0, 255).astype(np.uint8)

    img_ref = np.clip(img_original, 0, 255).astype(np.uint8)   

    psnr = peak_signal_noise_ratio(img_ref, img_geo_no_pad, data_range=255)
    ssim = structural_similarity(img_ref, img_geo_no_pad, data_range=255)

    score = 0.5 * psnr + 0.5 * (ssim * 100)
    print(f"→ PSNR: {psnr:.2f}, SSIM: {ssim:.4f}, Score: {score:.2f}")    

   
    return img_geo_no_pad, h_geo, psnr, ssim, score

### Novo

In [14]:
vetor = load_pickle("resultados_gaussian_256.pkl")
from bm3d import bm3d, BM3DProfile
from skimage.restoration import estimate_sigma
from skimage.color import rgb2gray
from skimage.filters import median
from skimage.morphology import disk


array_gnlm_bm3d_256_filtereds = []
for array in vetor:

    img_noisse_gaussian_np =array['img_noisse_gaussian_np']
    
    nlm_h =array['nlm_h']
    file_name =array['file_name']
    psnr_nlm =array['psnr_nlm']
    ssim_nlm =array['ssim_nlm']
    score_nlm =array['score_nlm']
    estimated_sigma_gaussian = array['estimated_sigma_gaussian']

    print(f'filename: {file_name}')
    print(f'psnr_nlm: {psnr_nlm}')
    print(f'ssim_nlm: {ssim_nlm}')
    print(f'score_nlm: {score_nlm}')

    dir_images = ("/workspace/ProjetoDoutorado/images")
    
    img = skimage.io.imread(f'{dir_images}/{file_name}')
    img = img[0, :, :] if len(img.shape) > 2 else img
    if len(img.shape) > 2:
        img = skimage.color.rgb2gray(img)
        img = 255 * img

    img_downscale = downscale(img)

    # Parâmetros
    f = 4
    t = 7
    nn = 10

    ini = time.time()

    print("\n--- Executando Pipeline 512x512 com mult = 1.55 ---")
    mult = 1.55
    img_filtered_gnlm, h_gnlm, psnr_gnlm, ssim_gnlm, score_gnlm = run_geonlm_pipeline(img_downscale, nlm_h, img_noisse_gaussian_np, f, t, mult, nn)   
       
    end = time.time()
    time_geonlm = end - ini
    skimage.io.imsave(f'{dir_out_geonlm}/{file_name}', img_filtered_gnlm.astype(np.uint8))

    img_bm3d = img_downscale.copy()

    if len(img_bm3d.shape) > 2:
        img_bm3d = skimage.color.rgb2gray(img_bm3d)
        img_bm3d = 255 * img_bm3d
    # Remove frames extras, se existirem (e.g. GIF com shape (1, H, W) ou (N, H, W, 3))
    if img_bm3d.ndim == 4:
        img_bm3d = img_bm3d[0]
    elif img_bm3d.ndim == 3 and img_bm3d.shape[2] != 3:
        img_bm3d = np.squeeze(img_bm3d)

    # Converte para escala de cinza, se necessário
    if img_bm3d.ndim == 3 and img_bm3d.shape[2] == 3:
        img_bm3d_gray = rgb2gray(img_bm3d)  # retorna float64 em [0, 1]
    else:
        img_bm3d_gray = img_bm3d.astype(np.float32) / 255.0  # já está em cinza
     # Agora está garantidamente em escala de cinza float [0,1]
    img_bm3d_gray = np.clip(img_bm3d_gray * 255, 0, 255).astype(np.uint8)
    
     # 1. Normaliza a imagem ruidosa para intervalo [0, 1]
    ruidosa_normalizada = img_noisse_gaussian_np.astype(np.float32) / 255.0

    sigma_est = estimate_sigma(ruidosa_normalizada, channel_axis=None)

    inicio = time.time()
    # 3. Define o perfil padrão do BM3D
    perfil_bm3d = BM3DProfile()
    fim = time.time() 
    time_bm3d = fim - inicio

    # 4. Aplica o BM3D com os argumentos definidos
    denoised = bm3d(
        ruidosa_normalizada,
        sigma_psd=sigma_est,
        profile=perfil_bm3d
    )
    denoised_sq = np.squeeze(denoised)   

    skimage.io.imsave(f'{dir_out_bm3d}/{file_name}',  np.clip(denoised_sq * 255, 0, 255).astype(np.uint8))

    
    psnr_bm3d = peak_signal_noise_ratio(img_bm3d_gray, (denoised * 255).astype(np.uint8))
    ssim_bm3d = structural_similarity(img_bm3d_gray, (denoised * 255).astype(np.uint8))
    score_bm3d = 0.5 * psnr_bm3d + 0.5 * (ssim_bm3d * 100)    
   

    dict = {
        #'img_filtered_gnlm':img_filtered_gnlm,
        'nlm_h':nlm_h, 
        'h_gnlm':h_gnlm,
        'estimated_sigma_gaussian':estimated_sigma_gaussian, 

        'ssim_nlm':ssim_nlm,
        'ssim_gnlm':ssim_gnlm,
        'ssim_bm3d': ssim_bm3d,

        'psnr_nlm': psnr_nlm,
        'psnr_gnlm':psnr_gnlm,
        'psnr_bm3d': psnr_bm3d,
        
        'score_nlm':score_nlm,   
        'score_gnlm':score_gnlm,  
        'score_bm3d': score_bm3d,

        'time_geonlm':time_geonlm,
        'time_bm3d':time_bm3d,
       
        'file_name':file_name,

    }
    array_gnlm_bm3d_256_filtereds.append(dict)
save_pickle(array_gnlm_bm3d_256_filtereds, "resultados_gaussian_gnlm_bm3d_256.pkl")

filename: 36.gif
psnr_nlm: 30.30409590052232
ssim_nlm: 0.8987952895783079
score_nlm: 60.09181242917656

--- Executando Pipeline 512x512 com mult = 1.55 ---

Executando GEONLM com h = 141.05 (base 91 * 1.55)
img_n.shape: (272, 272)
M: 264, N: 264


/opt/conda/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


→ PSNR: 31.56, SSIM: 0.9176, Score: 61.66
filename: 39.gif
psnr_nlm: 33.3957490295972
ssim_nlm: 0.8954283516775342
score_nlm: 61.46929209867531

--- Executando Pipeline 512x512 com mult = 1.55 ---

Executando GEONLM com h = 139.50 (base 90 * 1.55)
img_n.shape: (272, 272)
M: 264, N: 264
→ PSNR: 34.04, SSIM: 0.8936, Score: 61.70
filename: 4.gif
psnr_nlm: 30.964457630186793
ssim_nlm: 0.8526738892001923
score_nlm: 58.115923275103015

--- Executando Pipeline 512x512 com mult = 1.55 ---

Executando GEONLM com h = 133.30 (base 86 * 1.55)
img_n.shape: (272, 272)
M: 264, N: 264
→ PSNR: 31.91, SSIM: 0.8654, Score: 59.22
filename: 3.gif
psnr_nlm: 32.05051883945676
ssim_nlm: 0.877607480345437
score_nlm: 59.905633437000226

--- Executando Pipeline 512x512 com mult = 1.55 ---

Executando GEONLM com h = 130.20 (base 84 * 1.55)
img_n.shape: (272, 272)
M: 264, N: 264
→ PSNR: 33.05, SSIM: 0.8870, Score: 60.87
filename: 32.gif
psnr_nlm: 30.00196672947608
ssim_nlm: 0.8487767243550005
score_nlm: 57.4398195

In [15]:
salvar_resultados_em_xlsx(array_gnlm_bm3d_256_filtereds, 'saida_gnlm_bm3d_256_filtereds.xlsx')

Arquivo salvo em: saida_gnlm_bm3d_256_filtereds.xlsx


/tmp/ipykernel_272233/1866010612.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: float(x) if isinstance(x, np.float64) else x)
